In [1]:
import pandas as pd
import json
import requests
from quickchart import QuickChart
import gspread
import df2gspread as d2g
import pygsheets

gc = pygsheets.authorize(service_file='yourCreds.json')
sh = gc.open('yourSheet')
state = sh[0]

In [2]:
url = "https://api.covid19india.org/csv/latest/state_wise_daily.csv"
df = pd.read_csv(url).drop(['Date_YMD'],axis = 1)
df2 = df[['Date','Status','WB','TN','TG','AP','RJ','AS','BR','GA','GJ','HR','HP','JK','KA',
          'KL','MP','MH','OR','UP']]

In [3]:
stateDf = pd.DataFrame(columns = ["State Graph"])

In [4]:
stateList = ['WB','TN','TG','AP','RJ','AS','BR','GA','GJ','HR','HP','JK','KA',
             'KL','MP','MH','OR','UP']

for i in range(len(stateList)):
    
    stateName = stateList[i]
    
    active = []
    deaths = []
    recoveries = []

    active = df2[stateName].iloc[-30::3]
    recoveries = df2[stateName].iloc[-29::3]
    deaths = df2[stateName].iloc[-28::3]

    active = list(map(int,active))
    recoveries = list(map(int,recoveries))
    deaths = list(map(int,deaths))

    dateDict = {
        'Jan':1,
        'Feb':2,
        'Mar':3,
        'Apr':4,
        'May':5,
        'Jun':6,
        'Jul':7,
        'Aug':8,
        'Sep':9,
        'Oct':10,
        'Nov':11,
        'Dec':12
    }

    labelDate = []

    for i in range((len(df2)-30),len(df2),3):
        date = df2['Date'][i].split("-")
        date[1] = str(dateDict[str(date[1])])
        date_new = date[0] + "/" + date[1] + "/" + date[2]
        labelDate.append(date_new)

    quickchart_url = 'https://quickchart.io/chart/create'
    post_data = {'chart': {'type': 'line', 'data': {'labels': labelDate,
                 'datasets': [{'label': 'Cases', 
                               'data': active},
                             {'label': 'Deaths', 
                               'data': deaths},
                             {'label': 'Recoveries', 
                               'data': recoveries}
                             ]},
                          'options': {"title": {"display": "true",
                                      "text": 'COVID 19 ' + stateName + ' Active Cases Statistic'}}}}

    response = requests.post(
        quickchart_url,
        json = post_data,
    )

    if (response.status_code != 200):
        print('Error:', response.text)
    else:
        chart_response = json.loads(response.text)
        res = chart_response['url']
    
    stateDf = stateDf.append({"State Graph":res},ignore_index=True)


In [6]:
state.set_dataframe(stateDf,(0,3))